In [1]:
import os

In [2]:
%pwd

'c:\\Madhu\\2025_NLPDataScientist\\UKJobs\\Hiscox\\insuranceClaim\\End-to-end-MLOps-AWS-CICD-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Madhu\\2025_NLPDataScientist\\UKJobs\\Hiscox\\insuranceClaim\\End-to-end-MLOps-AWS-CICD-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from mlopsProject.constants import *
from mlopsProject.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation        
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])        

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            X_test_data_path = config.X_test_data_path,
            y_test_data_path = config.y_test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name           
        )

        return model_evaluation_config

In [13]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve 
from sklearn.metrics import roc_auc_score,roc_curve
from urllib.parse import urlparse
import numpy as np
import joblib

In [32]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_clf(true, predicted):
        acc = accuracy_score(true, predicted) # Calculate Accuracy
        f1 = f1_score(true, predicted) # Calculate F1-score
        precision = precision_score(true, predicted) # Calculate Precision
        recall = recall_score(true, predicted)  # Calculate Recall
        roc_auc = roc_auc_score(true, predicted) #Calculate Roc
        return acc, f1 , precision, recall, roc_auc
    

    def save_results(self):
        models_list = []
        accuracy_list = []
        auc= []

        model = joblib.load(self.config.model_path)

        test_x = pd.read_csv(self.config.X_test_data_path)
        test_y = pd.read_csv(self.config.y_test_data_path)
        
        predicted_qualities = model.predict(test_x)

        accuracy_sc = accuracy_score(test_y,predicted_qualities)
        cr = classification_report(test_y,predicted_qualities)

         # Saving metrics as local
        # scores = {"accuracy": accuracy_sc, "cr": cr}
        scores = {"accuracy": accuracy_sc}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [33]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-07-17 07:06:39,071: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-17 07:06:39,073: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-17 07:06:39,081: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-17 07:06:39,083: INFO: common: created directory at: artifacts]
[2024-07-17 07:06:39,085: INFO: common: created directory at: artifacts/model_evaluation]
[2024-07-17 07:06:39,256: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
